<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Turku_Finland_NASAPower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#pip install meteostat

In [34]:
import requests
import pandas as pd

# --- CONFIGURATION ---
# Turku, Finland
LAT = 60.4518
LON = 22.2666

# Dates (NASA POWER format: YYYYMMDD)
START_DATE = "20240101"
END_DATE = "20241231"

base_url = "https://power.larc.nasa.gov/api/temporal/hourly/point"

# Parameters
# T2M = Temp, WS10M = Wind, ALLSKY_SFC_SW_DWN = GHI
params = {
    "parameters": "T2M,WS10M,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF",
    "community": "RE", # Renewable Energy
    "longitude": LON,
    "latitude": LAT,
    "start": START_DATE,
    "end": END_DATE,
    "format": "JSON"
}

print("Fetching data from NASA POWER API...")

try:
    r = requests.get(base_url, params=params)

    if r.status_code == 200:
        data = r.json()

        # NASA returns data in a nested dictionary structure
        # Key: "T2M": { "2024010100": -5.2, "2024010101": -5.5 ... }
        properties = data['properties']['parameter']

        # We need to restructure this into a DataFrame
        # 1. Extract the timestamps (keys from one of the parameters)
        timestamps = sorted(properties['T2M'].keys())

        records = []
        for ts in timestamps:
            records.append({
                # Parse YYYYMMDDHH format
                'Time_Str': ts,
                'Temperature_C': properties['T2M'].get(ts),
                'Wind_Speed_ms': properties['WS10M'].get(ts),
                'GHI_Wm2': properties['ALLSKY_SFC_SW_DWN'].get(ts),
                'DNI_Wm2': properties['ALLSKY_SFC_SW_DNI'].get(ts),
                'DHI_Wm2': properties['ALLSKY_SFC_SW_DIFF'].get(ts)
            })

        df = pd.DataFrame(records)

        # Format Date/Time
        df['Time'] = pd.to_datetime(df['Time_Str'], format='%Y%m%d%H')
        df['Date'] = df['Time'].dt.strftime('%Y-%m-%d')
        df['Hour'] = df['Time'].dt.strftime('%H:%M')

        # Cleanup (NASA uses -999 for missing data)
        df = df.replace(-999, pd.NA)

        # Reorder
        cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2', 'DNI_Wm2', 'DHI_Wm2']
        df = df[cols]

        print("-" * 30)
        print(df.head())
        print("-" * 30)
        print(f"Total Rows: {len(df)}")

        filename = "Turku_NASA_POWER_2024.csv"
        df.to_csv(filename, index=False)
        print(f"Success! Saved to {filename}")

    else:
        print(f"Error {r.status_code}: {r.text}")

except Exception as e:
    print(f"Connection failed: {e}")

Fetching data from NASA POWER API...
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms  GHI_Wm2  DNI_Wm2  DHI_Wm2
0  2024-01-01  00:00         -17.84           2.08      0.0      0.0      0.0
1  2024-01-01  01:00         -17.92           2.21      0.0      0.0      0.0
2  2024-01-01  02:00         -18.03           2.33      0.0      0.0      0.0
3  2024-01-01  03:00         -18.40           2.47      0.0      0.0      0.0
4  2024-01-01  04:00         -18.90           2.65      0.0      0.0      0.0
------------------------------
Total Rows: 8784
Success! Saved to Turku_NASA_POWER_2024.csv
